In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

### Read and clean csv

In [2]:
df = pd.read_csv("las3.csv").drop(columns=["Timestamp", "Empty"]).rename(columns={
    "What is your name?": "name",
    "For each of the following, choose which days and times you are available (Please try to put at LEAST 10 available slots)": "times"
}).dropna()
df.head()

,name,times
0,Amy Clark (401),"Monday 1-2, Monday 2-3, Monday 3-4, Monday 4-5..."
1,Ruchi Sarkar (210),"Monday 10-11, Monday 1-2, Tues 10-11, Tues 11-..."
2,Dana Rubin (401),"Monday 10-11, Monday 1-2, Monday 4-5, Tues 1-2..."
3,Nidhi Murlidhar (401),"Monday 4-5, Tues 10-11, Tues 11-12, Tues 12-1,..."
4,Andrew Wortas (401),"Monday 10-11, Monday 1-2, Monday 2-3, Monday 3..."


In [3]:
all_times = set()

for index, row in df.iterrows():
    times = str(row["times"]).split(", ")
    for t in times:
        if t not in all_times:
            all_times.add(t)
            
all_times = list(all_times)

### Sort times

In [4]:
days = ["Monday", "Tues", "Wed", "Thur"]
time_order = ["10-11", "11-12", "12-1", "1-2", "2-3", "3-4", "4-5"]
sorted_times = []

for d in days:
    curr_day = [x for x in all_times if d in x]
    for t in time_order:
        for c in curr_day:
            if t in c:
                sorted_times.append(c)

### Create LA and Shift classes

In [5]:
class LA:
    
    def __init__(self, name):
        self.name = name
        self.hours_left = 8
        self.shifts = []
        self.possible_shifts = []
        
    def assign(self, shift):
        if self.hours_left > 0 and (shift not in self.shifts):
            self.hours_left -= 1
            self.shifts.append(shift)
            return True
        else:
            return False
        
    def addPossibleShift(self, shift):
        self.possible_shifts.append(shift)
        
    def __repr__(self):
        return "name: " + self.name + ", hours left: " + str(self.hours_left) + ", shifts: " + str([x.title for x in self.shifts])
        
        
class Shift:
    
    def __init__(self, title):
        self.title = title
        self.workers = []
        self.available_workers = []
        self.positions_remaining = 5
        
    def addLAToShift(self, la):
        if self.positions_remaining > 0 and (la not in self.workers):
            if la.assign(self):
                self.workers.append(la)
                self.positions_remaining -= 1
                return True
        else:
            return False
        
    def addAvailableWorker(self, la):
        self.available_workers.append(la)
        
    def __repr__(self):
        return "Shift: " + self.title + ", LAs assigned: " + str([x.name for x in self.workers]) + \
               ", positions left: " + str(self.positions_remaining) + ", available las: " + \
                str([x.name for x in self.available_workers])

### Add time columns

In [6]:
LAs = []
shifts = []

for t in sorted_times:
    df[t] = 0
    shifts.append(Shift(t))
    
for index, row in df.iterrows():
    l = LA(row["name"])
    for t in sorted_times:
        if t in str(row["times"]).split(", "):
            df[t][index] = 1
            l.addPossibleShift(t)
            [x for x in shifts if x.title == t][0].addAvailableWorker(l)
    
    LAs.append(l)

### Sorts shift times based on LAs available

In [7]:
workers = {}

for t in sorted_times:
    workers[t] = sum(df[t])

# Sort by fewest LAs available
sorted_workers = {k: v for k, v in sorted(workers.items(), key=lambda item: item[1])}
sorted_workers

{'Monday 12-1': 2,
 'Monday 11-12': 3,
 'Wed 11-12': 3,
 'Wed 12-1': 3,
 'Thur 2-3': 3,
 'Thur 3-4': 3,
 'Tues 2-3': 4,
 'Tues 3-4': 4,
 'Thur 4-5': 4,
 'Tues 10-11': 5,
 'Tues 1-2': 5,
 'Tues 4-5': 5,
 'Wed 3-4': 5,
 'Thur 10-11': 5,
 'Thur 12-1': 5,
 'Thur 1-2': 5,
 'Monday 10-11': 6,
 'Monday 3-4': 6,
 'Tues 12-1': 6,
 'Wed 4-5': 6,
 'Monday 1-2': 7,
 'Wed 10-11': 7,
 'Thur 11-12': 7,
 'Monday 2-3': 8,
 'Monday 4-5': 8,
 'Tues 11-12': 8,
 'Wed 1-2': 8,
 'Wed 2-3': 8}

### Assigns shifts

In [8]:
runs_since_changed = 0

while not runs_since_changed >= 29:
    for t in sorted_times:
        current_shift = [x for x in shifts if x.title == t][0]
        for l in current_shift.available_workers:
            if current_shift.addLAToShift(l):
                runs_since_changed = 0
                break
    runs_since_changed += 1

### Create HTML file

In [22]:
full_days = {
    "Monday": "Monday",
    "Tues": "Tuesday",
    "Wed": "Wednesday",
    "Thur": "Thursday",
}

html = """<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
        integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">

    <title>Home</title>
</head>

<body>

    <div class="container" style="margin-top:30px">
    
        <a href="altView.html">View shifts by LA</a>
        
        <br>

        <h1>Monday:</h1>
        <div class="row">"""
        

cols = 0
day_index = 0

for s in shifts:
        
    if days[day_index] not in s.title:
        day_index += 1
        html += "</div>\n"
        html += "<br><h1>" + full_days[days[day_index]] + ":</h1>"
        html += "<div class=\"row\">"
        cols = 0
        
    time = s.title.split(" ")[1]
    hour = time.split("-")[0]
        
    html += """<div class="col-3">
                   <div class="card border-dark mb-3" style="max-width: 18rem;">
                       <div class="card-header">""" + str(time + " am" if int(hour) == 10 else time + " pm") + """</div>
                       <div class="card-body text-dark">
                           <ul>"""
    
    for l in s.workers:
        html += "<li><p class=\"card-text\">" + l.name + "</p></li>"

    html += """             </ul>
                       </div>
                   </div>
               </div>"""
    cols += 1
            
html +="""</div>
    </div>

    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.4.1.slim.min.js"
        integrity="sha384-J6qa4849blE2+poT4WnyKhv5vZF5SrPo0iEjwBvKU7imGFAV0wwj1yYfoRSJoZ+n"
        crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js"
        integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo"
        crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"
        integrity="sha384-wfSDF2E50Y2D1uUdj0O3uMBJnjuUD4Ih7YwaYd1iqfktj0Uod8GCExl3Og8ifwB6"
        crossorigin="anonymous"></script>
</body>

</html>"""

file = open("index.html", "w")
file.write(html)

16923

In [18]:
html = """<!doctype html>
<html lang="en">

<head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css"
        integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">

    <title>LA View</title>
</head>

<body>

    <div class="container" style="margin-top:30px">
    
        <a href="index.html">Home</a>
        <br>
        <br>"""
        
for l in LAs:
    html += "<h3>" + l.name + "</h3>"
    html += "<ul>"
    for t in sorted_times:
        for s in l.shifts:
            if s.title == t:
                time = s.title.split(" ")[1]
                hour = time.split("-")[0]
                html += "<li>" + full_days[s.title.split(" ")[0]] + ": " + str(time + " am" if int(hour) == 10 else time + " pm") + "</li>"

    html += "</ul>"
    
html += """</div>

    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.4.1.slim.min.js"
        integrity="sha384-J6qa4849blE2+poT4WnyKhv5vZF5SrPo0iEjwBvKU7imGFAV0wwj1yYfoRSJoZ+n"
        crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js"
        integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo"
        crossorigin="anonymous"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"
        integrity="sha384-wfSDF2E50Y2D1uUdj0O3uMBJnjuUD4Ih7YwaYd1iqfktj0Uod8GCExl3Og8ifwB6"
        crossorigin="anonymous"></script>
</body>

</html>"""

file = open("altView.html", "w")
file.write(html)

4641